In [4]:
print("hello")

hello


In [1]:
import os
import cv2
import numpy as np

from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from xgboost import XGBClassifier


In [4]:
from skimage.feature import graycomatrix, graycoprops

def preprocess_image(img_path):
    img = cv2.imread(img_path)
    img = cv2.resize(img, (256, 256))
    blurred = cv2.GaussianBlur(img, (5, 5), 0)
    return img, blurred

def segment_disease_region(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    return thresh

def extract_color_features(image):
    mean = np.mean(image, axis=(0, 1))
    std = np.std(image, axis=(0, 1))
    return mean.tolist() + std.tolist()

def extract_texture_features(gray_img):
    glcm = graycomatrix(gray_img, [1], [0], symmetric=True, normed=True)
    return [
        graycoprops(glcm, 'contrast')[0, 0],
        graycoprops(glcm, 'correlation')[0, 0],
        graycoprops(glcm, 'energy')[0, 0],
        graycoprops(glcm, 'homogeneity')[0, 0]
    ]

def extract_shape_features(binary_img):
    contours, _ = cv2.findContours(binary_img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        cnt = max(contours, key=cv2.contourArea)
        area = cv2.contourArea(cnt)
        perimeter = cv2.arcLength(cnt, True)
    else:
        area, perimeter = 0, 0
    return [area, perimeter]


In [5]:
mobilenet = MobileNetV2(weights="imagenet", include_top=False, pooling="avg", input_shape=(224, 224, 3))


In [6]:
data_dir = "./Dataset"
selected_classes = []
all_features = []
labels = []
for label in os.listdir(data_dir):
    if selected_classes and label not in selected_classes:
        continue

    class_dir = os.path.join(data_dir, label)
    count = 0

    for fname in tqdm(os.listdir(class_dir), desc=f'Processing {label}'):
        
        fpath = os.path.join(class_dir, fname)

        try:
            orig, preprocessed = preprocess_image(fpath)
            gray = cv2.cvtColor(preprocessed, cv2.COLOR_BGR2GRAY)
            segment = segment_disease_region(preprocessed)

            # Handcrafted features
            color = extract_color_features(preprocessed)
            texture = extract_texture_features(gray)
            shape = extract_shape_features(segment)

            handcrafted_features = color + texture + shape

            # CNN features
            cnn_img = cv2.resize(orig, (224, 224))
            cnn_img = img_to_array(cnn_img)
            cnn_img = preprocess_input(cnn_img)
            cnn_features = mobilenet.predict(np.expand_dims(cnn_img, axis=0), verbose=0)[0]

            # Combine both
            combined = np.concatenate([handcrafted_features, cnn_features])
            all_features.append(combined)
            labels.append(label)
            count += 1

        except Exception as e:
            continue


Processing Cherry_(including_sour)___healthy: 100%|██████████| 854/854 [02:22<00:00,  5.98it/s]
Processing Cherry_(including_sour)___Powdery_mildew: 100%|██████████| 1052/1052 [02:33<00:00,  6.87it/s]
Processing Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot: 100%|██████████| 513/513 [01:18<00:00,  6.51it/s]
Processing Grape___healthy: 100%|██████████| 423/423 [01:02<00:00,  6.81it/s]
Processing Grape___Leaf_blight_(Isariopsis_Leaf_Spot): 100%|██████████| 1076/1076 [02:34<00:00,  6.98it/s]
Processing Orange___Haunglongbing_(Citrus_greening): 100%|██████████| 5507/5507 [1:07:04<00:00,  1.37it/s]     
Processing Tomato___Septoria_leaf_spot: 100%|██████████| 1771/1771 [09:12<00:00,  3.21it/s]   
Processing Tomato___Spider_mites Two-spotted_spider_mite: 100%|██████████| 1676/1676 [02:14<00:00, 12.47it/s]
Processing Tomato___Tomato_mosaic_virus: 100%|██████████| 373/373 [00:30<00:00, 12.13it/s]
Processing Tomato___Tomato_Yellow_Leaf_Curl_Virus: 100%|██████████| 5357/5357 [07:14<00:00, 1

In [8]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(labels)

X = np.array(all_features)
y = np.array(y_encoded)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)


In [11]:
learning_rates = [0.3 * (0.95 ** i) for i in range(10)]
model = XGBClassifier(
    n_estimators=100,
 #   learning_rate=0.2,
    max_depth=5,
    use_label_encoder=False,
    eval_metric='mlogloss'
)
for i, lr in enumerate(learning_rates):
    print(f"Training round {i+1} with learning rate = {lr:.5f}")
    
    model.set_params(learning_rate=lr)  # Update learning rate
    
    model.fit(
        X_train, y_train,
        xgb_model=model.get_booster() if i > 0 else None,  # Continue from previous model
        eval_set=[(X_test, y_test)],
        verbose=False
    )
#model.fit(X_train, y_train)


Training round 1 with learning rate = 0.30000


c:\Users\Nidhi Goel\Desktop\plant_disease\venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [14:41:32] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Training round 2 with learning rate = 0.28500


c:\Users\Nidhi Goel\Desktop\plant_disease\venv\Lib\site-packages\xgboost\core.py:1853: UserWarning: [18:20:30] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  state = model_file.__getstate__()
c:\Users\Nidhi Goel\Desktop\plant_disease\venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [18:20:30] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Training round 3 with learning rate = 0.27075


...

c:\Users\Nidhi Goel\Desktop\plant_disease\venv\Lib\site-packages\xgboost\core.py:1853: UserWarning: [18:48:44] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  state = model_file.__getstate__()
c:\Users\Nidhi Goel\Desktop\plant_disease\venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [18:48:47] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Training round 9 with learning rate = 0.19903


c:\Users\Nidhi Goel\Desktop\plant_disease\venv\Lib\site-packages\xgboost\core.py:1853: UserWarning: [19:01:23] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  state = model_file.__getstate__()
c:\Users\Nidhi Goel\Desktop\plant_disease\venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [19:01:24] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Training round 10 with learning rate = 0.18907


c:\Users\Nidhi Goel\Desktop\plant_disease\venv\Lib\site-packages\xgboost\core.py:1853: UserWarning: [19:05:06] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  state = model_file.__getstate__()
c:\Users\Nidhi Goel\Desktop\plant_disease\venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [19:05:08] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [12]:
y_pred = model.predict(X_test)
y_pred_labels = label_encoder.inverse_transform(y_pred)
y_test_labels = label_encoder.inverse_transform(y_test)
print("Accuracy:", accuracy_score(y_test_labels, y_pred_labels))
print("\nClassification Report:\n", classification_report(y_test_labels, y_pred_labels))
print("\nConfusion Matrix:\n", confusion_matrix(y_test_labels, y_pred_labels))

Accuracy: 0.9457742588841834

Classification Report:
                                                     precision    recall  f1-score   support

                                Apple___Apple_scab       0.91      0.90      0.91       126
                                 Apple___Black_rot       0.92      0.94      0.93       124
                          Apple___Cedar_apple_rust       0.94      0.80      0.86        55
                                   Apple___healthy       0.96      0.98      0.97       329
                               Blueberry___healthy       0.97      0.99      0.98       300
          Cherry_(including_sour)___Powdery_mildew       0.95      0.96      0.96       210
                 Cherry_(including_sour)___healthy       0.98      0.98      0.98       171
Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot       0.83      0.67      0.74       103
                       Corn_(maize)___Common_rust_       0.97      0.97      0.97       238
               Corn_(maiz

In [14]:
from sklearn.metrics import classification_report

report = classification_report(X_test, y_pred, output_dict=True)


ValueError: Classification metrics can't handle a mix of continuous-multioutput and multiclass targets

In [15]:
import matplotlib


In [16]:
import matplotlib.pyplot as plt
print("matplotlib is installed and working!")

matplotlib is installed and working!


In [18]:
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
model = XGBClassifier(
    n_estimators=100,
    learning_rate=0.2,
    max_depth=5,
    use_label_encoder=False,
    eval_metric='mlogloss'
)
model.fit(
    X_train,
    y_train,
    eval_set=[(X_test, y_test)],
    verbose=False  # Set to True if you want per-epoch output
)

results = model.evals_result()

# Plot the log loss over boosting rounds
train_loss = results['train']['mlogloss']
eval_loss = results['eval']['mlogloss']
epochs = range(len(train_loss))

# Plot the log loss over boosting rounds
plt.figure(figsize=(8, 5))
plt.plot(epochs, train_loss, label='Training Log Loss')
plt.plot(epochs, eval_loss, label='Validation Log Loss')
plt.xlabel('Boosting Rounds (Epochs)')
plt.ylabel('Log Loss')
plt.title('XGBoost Training and Validation Log Loss')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


c:\Users\Nidhi Goel\Desktop\plant_disease\venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [21:17:09] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


KeyError: 'train'

In [ ]:
import pickle
with open("disease_all_model.pkl", "rb") as file:
    data = pickle.load(file)

print(data)